# LInking Process

___
공부한 자료:
https://modoocode.com/135 씹어먹는 c++, 학교 강의자료.
___
</br>

### 링킹 전

`gcc -c <sourcecode.c>` 한 줄이 다음의 과정을 담는다.

1) 소스코드
`(.c)` 

2) 전처리 과정 
`(소스코드를 작은 단위로 쪼개면서, 해석할 수 있는 형태로 만듦 + 컴파일러 명령어 읽음)`

3) 컴파일 과정 
`(컴파일러가 하나의 소스파일 당 하나의 해석단위 (TU - Translation Unit)로 만듦 + (.s)어셈블리로 바꿈)`

4) 어셈블 과정 
`(어셈블러가 어셈블리를 대응하는 기계어로 바꿈)`

5) 목적코드
`(.o)`

간단히하고, 주제인 링킹단계로 넘어간다.
___
</br>

### 링킹단계

목적코드(.o)들을 모으고, 외부라이브러리 파일들과 한데 모아 PE(.exe) /ELF와 같은 실행파일을 만든다.

실행 파일은 하드디스크와 같은 외부저장장치에서, 실행시 메모리에 로드(load)된다.

</br>

- 프로그램은 메모리에 명령어들의 집합인 .text section(mov \\$s1, \\$s2 등)과 데이터(초기화된 static duration 변수 공간)집합인 .data section 등으로    
나뉘어 로드(load)된다.

    - 각 section 안에서도 각각의 객체(함수, 변수 등등)들에게는 Symbol이라는 고유의 이름이 붙어있다. 이를테면 다음과 같다.

```c
/* t1.c */

int y=16;
void fun1(){
    int x=1;
    static int z=1;
}

int main(){
    fun1();
    return 0;
}
```
위 소스코드를 `gcc -c t1.c`로  `t1.o` 이라는 목적 파일로 만들어보자.   
- `objdump -Sr t1.o`를 사용해 해당 목적파일의 .text section 내용을 살펴본다.

```

t1.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <fun1>:
   0:	f3 0f 1e fa          	endbr64 
   4:	55                   	push   %rbp
   5:	48 89 e5             	mov    %rsp,%rbp
   8:	c7 45 fc 01 00 00 00 	movl   $0x1,-0x4(%rbp)
   f:	90                   	nop
  10:	5d                   	pop    %rbp
  11:	c3                   	retq   

0000000000000012 <main>:
  12:	f3 0f 1e fa          	endbr64 
  16:	55                   	push   %rbp
  17:	48 89 e5             	mov    %rsp,%rbp
  1a:	b8 00 00 00 00       	mov    $0x0,%eax
  1f:	e8 00 00 00 00       	callq  24 <main+0x12>
			20: R_X86_64_PLT32	fun1-0x4
  24:	b8 00 00 00 00       	mov    $0x0,%eax
  29:	5d                   	pop    %rbp
  2a:	c3                   	retq 

```

--> 전역함수 fun1()은 .text secion에서, fun1이라는 symbol을 가지게 된다. main도 마찬가지이다.



.data section에 있을 변수들(변수 저장기간 - 유지기간?이 *static*인 변수들)도 찾아본다.

- `gcc objdump -s t1.o`

```
Contents of section .data:
 0000 10000000 01000000                    ........  

```

--> 16진수(0x)로 1과 16이 (리틀엔디안으로 읽는다)들어있다. 각각 static int인 z와, 전역변수 int y의 값이다.



#### Symbol ?
symbol에 대해 조금 더 알아본다.   


c에서 심볼이란 데이터 고유의 식별자(고유한 이름)으로, 코드를 읽는 사람에게 편리함을 제공한다!
- 주소값 대신 심볼(이름)을 부르는 것이다.
이를 위해 Symbol Table이라는 표가 있어 symbol(이름)과 주소를 매칭해준다.
> ~~앙증맞은 표가 그려지네...~~
|symbol|주소|
|------|---|
|x|0x110|
|y|0x120|
|z|0x123|
|fun1|0x2124|
|main|0x0000|

symbol table은 compiler가 만들어서 목적파일(.o)에 넣어준다.


#### 목적 파일의 Section

> 참고.
> 1. word size, byte ordering, file type, machine type...   
2.  page size, virtual addr memory seg., seg sizes...   
3. code   
4. read only data - jump tables   
5. 초기화된 global var.   
6. 초기화 x인 global var.    
7. symbol table (static var. names, section names & location)   
8. relative info for .text sec. (addr of instructions that will need to be modified in the exe.)   
9. rel. info for.data sec. (addr of pointer data that will need to be modified)   
10. info. for symbolic debugging(gcc -g)   
11. offsets& sizes of each section.

|1|ELF header|
|---|---|
|2|segment header table|
|3|.text section|
|4|.rodata section|
|5|.data section|
|6|.bss section|
|7|.symtab section|
|8|.rel.text section|
|9|.rel.data section|
|10|.debug section|
|11|section header table|

___

</br></br>
symbol앞의 숫자는 메모리상의 주솟값인가?

</br>

1. 실제 물리 메모리가 아닌, **가상 메모리 상에 올릴 주솟값**이다.

> - TU마다 페이지page라고 부르는 단위로 각각 page table을 가진다.
> - page table은 후에 실제 물리 메모리상의 주솟값으로 매칭해준다.

2. 이마저도, 가상 메모리 상에서 어떻게 load할 지는 정해지지 않았기에, 링커가 해결해줘야 한다.

> TU로부터 생성한 목적코드들에는 각각의 symbol들의 
> - 저장 방식 (변수 선언/ 정의 시 붙여주는 명령어 - static, extern 등)
> - 링크방식  (R_x86_64_plt32, 등등)
>  
에 따른 **희망위치**만 적혀있다.

</br>
`objdump`말고, 더 간단하게 `nm`명령어로도 볼 수 있다.

- `nm t1.o` 

```

0000000000000000 T fun1
0000000000000012 T main
0000000000000000 D y
0000000000000004 d z.1914

```
> 각 열별로 다음과 같이 읽는다.

|1열|2열|3열|
|---|---|---|
|각 section으로부터 해당 symbol까지 offset|section|symbol|

section이 대문자는 외부링크방식, 소문자는 내부링크방식이다. (각각, 외부에서도 사용가능, 내부에서만 사용가능)  



</br></br>
**링킹전까지 희망위치만 적는 이유:**   

애초에 각 섹션은 어디에 위치할 것이며, 다른 소스파일들로부터 온 목적파일끼리는 각자의 섹션들을 어떻게 배치할지 모른다.

> **더 최악은, 목적파일(.o)에 희망위치 조차도 적지 못하는 경우가 있다.**
> - 외부에서 들고오는 extern 변수들, include 함수들은? 


### Relocation

- 링커는 분리된 code/ data section들을 각각 하나의 섹션으로 모은다.
- 목적파일(.o)에서 상대적 위치로 나타나있던 symbol들을 실행파일에서 절대적 위치(메모리주소)로 나타낸다.
    - symbol table을 갱신해서 symbol의 새로운 위치를 나타내게 한다. 
- symbol들의 위치가 확정되면, 재배치 테이블(rel. table)에 따라서 바꿔야할 값들을 바꿔준다.

예로, static 변수 `x` 의 위치를 .data section이나 bss section의 위치를 모르기에, 추후에 재배치 되도록 compiler가 재배치 테이블에 기록해 둔다.

예시 프로그램을 살펴보자.
우선, 아래 두개의 소스코드가 있다. </br>

```c

/* t11.c */

#include "t12.h"

int func(){};
extern int x;

int main(){
        func1();
        static int  y=1;
        char z='A';
        x=x+y;
        return 0;
}

```
```c

/* t12.h */

int func1(){};

int x=30;                               

```

</br>

`gcc -c <name.c>` 로 .c 파일을 컴파일한다. 

- `nm t11.o`

```
000000000000000b T func
0000000000000000 T func1
0000000000000016 T main
0000000000000000 D x
0000000000000004 d y.1918

```
</br>
t11의 목적파일을 읽어보자.    

- T는 .text section, D는 .data section이다. func1과 x란 symbol이 있다.
- fun1, x는 외부링크방식으로써 (대문자알파벳), 위치 offset값이 0이라 나와있다. (위치를 0x0으로 뒀다.)


위에서 언급했듯, extern으로 가져온 x, 다른 header file에서 (예시의 경우에는 t12.h) 가져온 함수 func1()이기 때문이다.


</br>

즉, **외부에서 가져온 함수라 compiler는 희망위치조차 적지 못한다.**
- 함수 코드도 모르고, 사이즈도 몰라서 그런 것같다.

</br></br>
**재배치(Relocation)가 필요하다.**
</br>

- `objdump -Sr t11.o`

```
t11.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <func1>:
   0:	f3 0f 1e fa          	endbr64 
   4:	55                   	push   %rbp
   5:	48 89 e5             	mov    %rsp,%rbp
   8:	90                   	nop
   9:	5d                   	pop    %rbp
   a:	c3                   	retq   

000000000000000b <func>:
   b:	f3 0f 1e fa          	endbr64 
   f:	55                   	push   %rbp
  10:	48 89 e5             	mov    %rsp,%rbp
  13:	90                   	nop
  14:	5d                   	pop    %rbp
  15:	c3                   	retq   

0000000000000016 <main>:
  16:	f3 0f 1e fa          	endbr64 
  1a:	55                   	push   %rbp
  1b:	48 89 e5             	mov    %rsp,%rbp
  1e:	48 83 ec 10          	sub    $0x10,%rsp
  22:	b8 00 00 00 00       	mov    $0x0,%eax
  27:	e8 00 00 00 00       	callq  2c <main+0x16>
			28: R_X86_64_PLT32	func1-0x4
  2c:	c6 45 ff 41          	movb   $0x41,-0x1(%rbp)
  30:	8b 15 00 00 00 00    	mov    0x0(%rip),%edx        # 36 <main+0x20>
			32: R_X86_64_PC32	x-0x4
  36:	8b 05 00 00 00 00    	mov    0x0(%rip),%eax        # 3c <main+0x26>
			38: R_X86_64_PC32	.data
  3c:	01 d0                	add    %edx,%eax
  3e:	89 05 00 00 00 00    	mov    %eax,0x0(%rip)        # 44 <main+0x2e>
			40: R_X86_64_PC32	x-0x4
  44:	b8 00 00 00 00       	mov    $0x0,%eax
  49:	c9                   	leaveq 
  4a:	c3                   	retq   


```

위에서(.text section) main symbol을 보자. 쭉, 내려가다보면 offset이 0x28인 줄에서 특이한 부분이 보인다.
- `28: R_X86_64_PLT32	func1-0x4`
> PLT32는 우선 재배치(Relocation) 방식 중의 하나라고 알고가자.


또 내려가다보면 이번에는 offset이 32, 38, 40에서 R_X86_64_PLT32와 비슷한 이름이 보인다.
- `R_X86_64_PC32` (0x32 offset을 예로 들어보자.)
> R_X86_64_PC32는 0x32의 4byte를 S+A-P의 값으로 치환하라는 뜻이다. </br>(0x32는 위의 줄의 0x30에서 8b 15 다음으로 오게되는 메모리공간을 가리킨다. 현재는 0으로 채워져있다. 어떤 주소값일 지 모르기 때문이다.)</br></br>S는 재배치(link)
후 해당 심볼(외부함수 등, 0x0에서 바뀌게될 주소가 가리키는 심볼)위치, </br>A는 Addend, </br>P는 '재배치 후, 바꿔줘야할 부분(명령어 속 외부심볼주소) 위치'이다.     

    - Addend가 뭔지 몰라서 우리 GPT한테 물어봤다. https://platform.openai.com/playground

    > The addend is the value that is added to the original address to obtain the relocated address. **This ensures that the program or data item is correctly positioned in the new location.**</br></br>The addend ensures the correct position of the program or data item by adding the correct amount of offset to the original address. **This offset is calculated based on the size of the program or data item and the space available at the new location.** By adding the correct amount of offset, the program or data item is correctly positioned at the new location.

0x32위치에 바꿔들어갈 값을 직접 계산해 본다.

`gcc t11.o -o t11f
` 우선, 링킹을 해두고, 
</br>
1. S:    
- `nm t11f`

쭈욱 내리다보면,

```
0000000000004018 B __bss_start
0000000000004018 b completed.8061
                 w __cxa_finalize@@GLIBC_2.2.5
0000000000004000 D __data_start
0000000000004000 W data_start
0000000000001070 t deregister_tm_clones
00000000000010e0 t __do_global_dtors_aux
0000000000003df8 d __do_global_dtors_aux_fini_array_entry
0000000000004008 D __dso_handle
0000000000003e00 d _DYNAMIC
0000000000004018 D _edata
0000000000004020 B _end
00000000000011f8 T _fini
0000000000001120 t frame_dummy
0000000000003df0 d __frame_dummy_init_array_entry
000000000000217c r __FRAME_END__
0000000000001134 T func
0000000000001129 T func1
0000000000003fc0 d _GLOBAL_OFFSET_TABLE_
                 w __gmon_start__
0000000000002004 r __GNU_EH_FRAME_HDR
0000000000001000 t _init
0000000000003df8 d __init_array_end
0000000000003df0 d __init_array_start
0000000000002000 R _IO_stdin_used
                 w _ITM_deregisterTMCloneTable
                 w _ITM_registerTMCloneTable
00000000000011f0 T __libc_csu_fini
0000000000001180 T __libc_csu_init
                 U __libc_start_main@@GLIBC_2.2.5
000000000000113f T main
00000000000010a0 t register_tm_clones
0000000000001040 T _start
0000000000004018 D __TMC_END__
0000000000004010 D x
0000000000004014 d y.1918
```

맨 밑에서 두번째 줄, x는 offset 0x4010에 위치한 심볼이다.
따라서 S = 0x4010
</br></br>
2. A: 

A는 addend로, 위의 objdump -Sr t11.o을 했을 때, x-0x4를 보고도 알 수 있다.
readelf -r <name.o>로 재배치 테이블을 볼 수도 있다.

- `readelf -r t11.o`

```

Relocation section '.rela.text' at offset 0x2c8 contains 4 entries:
  Offset          Info           Type           Sym. Value    Sym. Name + Addend
000000000028  000a00000004 R_X86_64_PLT32    0000000000000000 func1 - 4
000000000032  000b00000002 R_X86_64_PC32     0000000000000000 x - 4
000000000038  000300000002 R_X86_64_PC32     0000000000000000 .data + 0
000000000040  000b00000002 R_X86_64_PC32     0000000000000000 x - 4

Relocation section '.rela.eh_frame' at offset 0x328 contains 3 entries:
  Offset          Info           Type           Sym. Value    Sym. Name + Addend
000000000020  000200000002 R_X86_64_PC32     0000000000000000 .text + 0
000000000040  000200000002 R_X86_64_PC32     0000000000000000 .text + b
000000000060  000200000002 R_X86_64_PC32     0000000000000000 .text + 16
```

x-4가 보인다.    
따라서 addend(A) = 0x-4
</br></br>
3. P:
</br>
x는 main symbol로부터 offset 0x1c에서 불러졌다.</br>
main symbol의 재배치 후 위치는, nm t11f에서 봤듯, 0x113f이다.</br>
0x113f + 0x1c = 0x115b.</br>
따라서 P = 0x115b
</br></br>

4.S + A - P = 0x4010 - 0x4 - 0x115b = 0x2eb1


이제 `objdump -s t11f`에서 해당부분이 0x2eb1로 바뀌었는지 확인한다.

```
t11f:     file format elf64-x86-64


Disassembly of section .init:

(생략)

0000000000001129 <func1>:
    1129:	f3 0f 1e fa          	endbr64 
    112d:	55                   	push   %rbp
    112e:	48 89 e5             	mov    %rsp,%rbp
    1131:	90                   	nop
    1132:	5d                   	pop    %rbp
    1133:	c3                   	retq   

0000000000001134 <func>:
    1134:	f3 0f 1e fa          	endbr64 
    1138:	55                   	push   %rbp
    1139:	48 89 e5             	mov    %rsp,%rbp
    113c:	90                   	nop
    113d:	5d                   	pop    %rbp
    113e:	c3                   	retq   

000000000000113f <main>:
    113f:	f3 0f 1e fa          	endbr64 
    1143:	55                   	push   %rbp
    1144:	48 89 e5             	mov    %rsp,%rbp
    1147:	48 83 ec 10          	sub    $0x10,%rsp
    114b:	b8 00 00 00 00       	mov    $0x0,%eax
    1150:	e8 d4 ff ff ff       	callq  1129 <func1>
    1155:	c6 45 ff 41          	movb   $0x41,-0x1(%rbp)
    1159:	8b 15 b1 2e 00 00    	mov    0x2eb1(%rip),%edx        # 4010 <x>
    115f:	8b 05 af 2e 00 00    	mov    0x2eaf(%rip),%eax        # 4014 <y.1918>
    1165:	01 d0                	add    %edx,%eax
    1167:	89 05 a3 2e 00 00    	mov    %eax,0x2ea3(%rip)        # 4010 <x>
    116d:	b8 00 00 00 00       	mov    $0x0,%eax
    1172:	c9                   	leaveq 
    1173:	c3                   	retq   
    1174:	66 2e 0f 1f 84 00 00 	nopw   %cs:0x0(%rax,%rax,1)
    117b:	00 00 00 
    117e:	66 90                	xchg   %ax,%ax

(생략)
```

1159:	8b 15 b1 2e 00 00    	mov    0x2eb1(%rip),%edx        # 4010 <x>
로 돼있는 걸 볼 수 있다.


`mov    0x2eb1(%rip),%edx`나 `1159:	8b 15 b1 2e 00 00`를 보면
바뀐 부분이 계산한 값과 같다.


</br></br>
#### R_X86_64_PLT32방식
- `28: R_X86_64_PLT32    func1-0x4`로 다시 가보자.

objdump -Sr t11.o 에서 분명 `27:    e8 00 00 00 00           callq  2c <main+0x16>` 였다.</br>
e8다음에 0x0부분을 R_X86_64_PLT32	방식으로 채워넣어야 한다.</br>
</br>
R_X86_64_PLT32은, 
> - 프로시져 링크 테이블 (**P**rocedure **L**inkage **T**able) 을 사용한 링킹 방식
> - 하지만, 동적링크를 사용하지 않았을 때는, R_X86_64_PC32 방식과 같다. 

확인해보자.
</br></br>

`objdump -Sr t11f`에서 해당 부분을 찾는다.

`1150:	e8 d4 ff ff ff       	callq  1129 <func1>` </br></br>
리틀엔디안으로 0xffffffd4는 -0x2c, %rip 레지스터는 다음명렁어인 0x1155를 가리키므로, 해당 명령어는 0x1129(=0x1155-0x2c) 주소의 내용을 불러온다.</br>
0x1129에는 다음과 같은 심볼이 있다.
> 0000000000001129 \<func1>:

**R_X86_64_PC32와 같다!**
> - R_X86_64_PC32는 S+A-P아닌가?
      - 직접 해보자.
      S: 0000000000001129 T func1. 따라서 0x1129.
      A: -0x4
      P: main(0x16)로부터 0x12(=0x28-0x16)떨어져있었다. 재배치 후, main은 0x113f이므로, 0x1151(=0x113f+0x12)이다.
      따라서 0x1129 -0x4 -0x1151= -0x2c. e8다음 0x0이 -0x2c로 채워져있어야 하므로, 맞다.
     